In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Admin\\Downloads\\spark-3.5.0-bin-hadoop3'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
    .builder
    .appName("ResourceAllocationApp")
    .master("spark://localhost:7077")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true")
    .config("spark.cores.max", "4")
    .config("spark.executor.memory", "2g")
    .config("spark.executor.cores", "2")
    .config("spark.executor.instance", "2")
    .config("spark.sql.adaptive.enabled", "false")
    .config("spark.dynamicAllocation.minExecutors", "0")
    .config("spark.dynamicAllocation.maxExecutors", "5")
    .config("spark.dynamicAllocation.schedulerBacklogTimeout", "1s")
    .config("spark.dynamicAllocation.executorIdleTimeout", "10s")
    .config("spark.dynamicallocation.cachedExecutorIdleTimeout", "10s")
    .getOrCreate()
)

sc = spark.sparkContext

spark

In [6]:
# Create schema for Yellow Taxi Data
 
taxiSchema = (
                    StructType
                    ([ 
                        StructField("VendorId"               , IntegerType()   , True),
                        StructField("lpep_pickup_datetime"   , TimestampType() , True),
                        StructField("lpep_dropoff_datetime"  , TimestampType() , True),                            
                        StructField("passenger_count"        , DoubleType()    , True),
                        StructField("trip_distance"          , DoubleType()    , True),
                        StructField("RatecodeID"             , DoubleType()    , True),                            
                        StructField("store_and_fwd_flag"     , StringType()    , True),
                        StructField("PULocationID"           , IntegerType()   , True),
                        StructField("DOLocationID"           , IntegerType()   , True),                            
                        StructField("payment_type"           , IntegerType()   , True),                            
                        StructField("fare_amount"            , DoubleType()    , True),
                        StructField("extra"                  , DoubleType()    , True),
                        StructField("mta_tax"                , DoubleType()    , True),
                        StructField("tip_amount"             , DoubleType()    , True),
                        StructField("tolls_amount"           , DoubleType()    , True),
                        StructField("improvement_surcharge"  , DoubleType()    , True),
                        StructField("total_amount"           , DoubleType()    , True),
                        StructField("congestion_surcharge"   , DoubleType()    , True),
                        StructField("airport_fee"            , DoubleType()    , True)
                    ])
               )

In [7]:
yellowTaxiDf = (
    spark
    .read
    .option("header", "true")
    .schema(taxiSchema)
    .csv("C:\DataFiles\YellowTaxis_202210.csv")
)
yellowTaxiGroupedDf = (
    yellowTaxiDf.dropDuplicates().groupBy("PULocationID").agg(sum("total_amount"))
)
yellowTaxiGroupedDf.show()

+------------+------------------+
|PULocationID| sum(total_amount)|
+------------+------------------+
|         148| 795087.6300000016|
|         243| 16739.69999999999|
|          31|            935.15|
|         137| 762800.5900000012|
|          85|3172.5600000000004|
|         251|            521.63|
|          65|          80190.23|
|         255| 51031.77999999999|
|          53|1601.2899999999995|
|         133|3281.4800000000023|
|          78|3559.9399999999996|
|         155| 5599.840000000005|
|         108|3688.3300000000004|
|         211| 529940.7899999999|
|         193|20420.559999999994|
|          34|           2659.96|
|         126|1586.9799999999993|
|         101|1735.7099999999994|
|         115|160.15000000000003|
|          81|2964.3900000000003|
+------------+------------------+
only showing top 20 rows

